# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


####  Run this cell to set up and start your interactive session.


In [1]:
%idle_timeout 2880
%glue_version 3.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.37.0 
Current idle_timeout is 2880 minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 3.0
Previous worker type: G.1X
Setting new worker type to: G.1X
Previous number of workers: 5
Setting new number of workers to: 5
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::393747608406:role/glue_role_redsfhit_project
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: 1ff8a66b-15d9-43b9-8b6e-361005fdd474
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.0
--enable-glue-datacatalog true
Waiting for session 1ff8

#### Example: Create a DynamicFrame from a table in the AWS Glue Data Catalog and display its schema


In [2]:
dyf = glueContext.create_dynamic_frame.from_catalog(database='redsfhift_data', table_name='btc')
dyf.printSchema()

root
|-- timestamp: long
|-- open: string
|-- high: string
|-- low: string
|-- close: string
|-- volume_(btc): string
|-- volume_(currency): string
|-- weighted_price: string


In [3]:
btc_dyf = ApplyMapping.apply(
    frame = dyf, 
    mappings = [
        ("timestamp","long","timestamp","Integer"), 
        ("open","string","open","double"), 
        ("high","string","high","double"), 
        ("low","string","low","double"), 
        ("close","string","close","double"),
        ("volume_(btc)","string","volume_(btc)","double"), 
        ("volume_(currency)","string","volume_(currency)","double"), 
        ("weighted_price","string","weighted_price","double")
    ],
    transformation_ctx = "btc_dyf_ctx"
)

#### Example: Convert the DynamicFrame to a Spark DataFrame and display a sample of the data


In [4]:
from datetime import datetime
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType,IntegerType,TimestampType
date_UDF=udf(lambda x:datetime.fromtimestamp(x),TimestampType()) 
year_UDF = udf(lambda x:x.year,IntegerType()) 
month_UDF=  udf(lambda x:x.month,IntegerType())   

In [5]:
df = btc_dyf.toDF()
df=df.withColumn("date",date_UDF(col("Timestamp")))
df=df.withColumn("Year",year_UDF(col("date")))
df=df.withColumn("Month",month_UDF(col("date")))
df = df.replace(float('nan'), None)
df.createOrReplaceTempView("table_df")
#df.printSchema()
df.show()

+----------+----+----+----+-----+------------+-----------------+--------------+-------------------+----+-----+
| timestamp|open|high| low|close|volume_(btc)|volume_(currency)|weighted_price|               date|Year|Month|
+----------+----+----+----+-----+------------+-----------------+--------------+-------------------+----+-----+
|1325317920|4.39|4.39|4.39| 4.39|  0.45558087|     2.0000000193|          4.39|2011-12-31 07:52:00|2011|   12|
|1325317980|null|null|null| null|        null|             null|          null|2011-12-31 07:53:00|2011|   12|
|1325318040|null|null|null| null|        null|             null|          null|2011-12-31 07:54:00|2011|   12|
|1325318100|null|null|null| null|        null|             null|          null|2011-12-31 07:55:00|2011|   12|
|1325318160|null|null|null| null|        null|             null|          null|2011-12-31 07:56:00|2011|   12|
|1325318220|null|null|null| null|        null|             null|          null|2011-12-31 07:57:00|2011|   12|
|

#### Example: Write the data in the DynamicFrame to a location in Amazon S3 and a table for it in the AWS Glue Data Catalog


In [6]:
r1=spark.sql("select  avg(Close), Year   from table_df where Close IS NOT NULL  group by 2 order by 2  ;")

In [11]:
r1.write.mode("overwrite").partitionBy("Year").csv("s3://redshift-tutorial-datasets/Glue_data/btc/btc_agg.csv")

In [18]:
from awsglue.dynamicframe import DynamicFrame
r2=spark.sql("select  sum(weighted_price), Year   from table_df where Close IS NOT NULL  group by 2 order by 2  ;")
dyf2=DynamicFrame.fromDF(r2, glueContext, "weighted_price")

In [21]:
glueContext.write_dynamic_frame.from_options(frame = dyf2, connection_type = "s3", format = "csv", connection_options = {"path": "s3://redshift-tutorial-datasets/Glue_data/btc/year_agg.csv ","partitionKeys": ['Year']}, transformation_ctx = "year_agg_trf")